# XML / HTML 데이터 사용하기

In [102]:
import pandas as pd
import urllib.request

# BeautifulSoup은 bs4 라는 패키지에 포함 된 모듈이다
from bs4 import BeautifulSoup

##### JSON과 같지만 type이 다르다

In [41]:
sampleURL="http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieList.xml?key=9a8b54edfc3a767c4644a74ea2305ba8&itemPerPage=40&openDt=2020"
sampleURL

'http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieList.xml?key=9a8b54edfc3a767c4644a74ea2305ba8&itemPerPage=40&openDt=2020'

In [42]:
res = urllib.request.urlopen(sampleURL)
res

In [43]:
soup = BeautifulSoup(res, "html.parser")
# 보기 좋게 출력하기
print(soup.prettify())

<?xml version="1.0" encoding="utf-8" standalone="yes"?>
<movielistresult>
 <totcnt>
  76577
 </totcnt>
 <movielist>
  <movie>
   <moviecd>
    20201803
   </moviecd>
   <movienm>
    소년 아메드
   </movienm>
   <movienmen>
    Young Ahmed
   </movienmen>
   <prdtyear>
    2019
   </prdtyear>
   <opendt>
   </opendt>
   <typenm>
    장편
   </typenm>
   <prdtstatnm>
    개봉예정
   </prdtstatnm>
   <nationalt>
    벨기에,프랑스
   </nationalt>
   <genrealt>
    드라마
   </genrealt>
   <repnationnm>
    벨기에
   </repnationnm>
   <repgenrenm>
    드라마
   </repgenrenm>
   <directors>
    <director>
     <peoplenm>
      장 피에르 다르덴
     </peoplenm>
    </director>
    <director>
     <peoplenm>
      뤽 다르덴
     </peoplenm>
    </director>
   </directors>
   <companys>
   </companys>
  </movie>
  <movie>
   <moviecd>
    20201863
   </moviecd>
   <movienm>
    외아들
   </movienm>
   <movienmen>
    The Only Son
   </movienmen>
   <prdtyear>
    1963
   </prdtyear>
   <opendt>
   </opendt>
   <typenm>
    장편
   </t

#### value 가져오기
##### xml은 소문자로 soup에 저장되었으니 주의

In [44]:
soup.find("moviecd")

<moviecd>20201803</moviecd>

In [45]:
soup.find("moviecd").text

'20201803'

In [46]:
soup.find_all("moviecd")

[<moviecd>20201803</moviecd>,
 <moviecd>20201863</moviecd>,
 <moviecd>20202001</moviecd>,
 <moviecd>20200828</moviecd>,
 <moviecd>20201122</moviecd>,
 <moviecd>20200601</moviecd>,
 <moviecd>20208182</moviecd>,
 <moviecd>20200836</moviecd>,
 <moviecd>20201341</moviecd>,
 <moviecd>20209882</moviecd>,
 <moviecd>20201322</moviecd>,
 <moviecd>20201323</moviecd>,
 <moviecd>20201965</moviecd>,
 <moviecd>20201321</moviecd>,
 <moviecd>20201964</moviecd>,
 <moviecd>20209004</moviecd>,
 <moviecd>20209005</moviecd>,
 <moviecd>20201381</moviecd>,
 <moviecd>20197848</moviecd>,
 <moviecd>20209426</moviecd>,
 <moviecd>20209507</moviecd>,
 <moviecd>20201040</moviecd>,
 <moviecd>20207564</moviecd>,
 <moviecd>20200444</moviecd>,
 <moviecd>20200766</moviecd>,
 <moviecd>20200787</moviecd>,
 <moviecd>20200773</moviecd>,
 <moviecd>20200675</moviecd>,
 <moviecd>20209722</moviecd>,
 <moviecd>20209921</moviecd>,
 <moviecd>20209464</moviecd>,
 <moviecd>20209225</moviecd>,
 <moviecd>20209488</moviecd>,
 <moviecd>

In [47]:
soup.find_all("moviecd")[1].text

'20201863'

In [48]:
soup.movielistresult.movielist.moviecd

<moviecd>20201803</moviecd>

In [62]:
# 영화이름, 영문이름, 개봉날짜, 1번감독이름, 영화코드값
print(soup.find_all("movie")[0].find("movienm").text)
print(soup.find_all("movie")[0].find("movienmen").text)
print(soup.find_all("movie")[0].find("opendt").text)
print(soup.find_all("movie")[0].find("director").text)
print(soup.find_all("movie")[0].find("moviecd").text)


소년 아메드
Young Ahmed

장 피에르 다르덴
20201803


### soup에 담긴 영화 정보들을 가지고
1) DF에 영화코드, 영화이름. 영문이름, 1번감독이름, 개봉날짜 담기(DF 완성)<br>
2) 개봉날짜를 바탕으로 lambda apply 활용해서 year/  month / day의 컬럼 추가 생성<br>
3) moivecd 컬럼을 DF의 reset_index를 활용햐서 인덱스로 설정<br>
4) 개별 영화에 대한 상세정보(장르, 1번배우, 1번배우 배역, 영화출연 배우 수)

In [107]:
movieIdDF = pd.DataFrame()
movieIdDF_temp = pd.DataFrame()
movieIdDF_temp = movieIdDF_temp.append({
    "movieCd":"",
    "movieNm":"",
    "movieNmEn":"",
    "openDt":"",
    "directorpeopleNm":""},
    ignore_index=True)

num = len(soup.find_all("movie"))
temp_movcd_idx = movieIdDF_temp.columns.get_loc("movieCd")
temp_movnm_idx = movieIdDF_temp.columns.get_loc("movieNm")
temp_movnmen_idx = movieIdDF_temp.columns.get_loc("movieNmEn")
temp_open_idx = movieIdDF_temp.columns.get_loc("openDt")
temp_dir_idx = movieIdDF_temp.columns.get_loc("directorpeopleNm")

for i in range(0,num):
    index=soup.find_all("movie")[i]
    movieIdDF_temp.iat[0,temp_movcd_idx]=index.find("moviecd").text
    movieIdDF_temp.iat[0,temp_movnm_idx]=index.find("movienm").text
    movieIdDF_temp.iat[0,temp_movnmen_idx]=index.find("movienmen").text
    movieIdDF_temp.iat[0,temp_open_idx]=index.find("opendt").text
    if index.find("director"):
        movieIdDF_temp.iat[0,temp_dir_idx]=index.find("director").find("peoplenm").text
    else:
        movieIdDF_temp.iat[0,temp_dir_idx]=""
    movieIdDF = movieIdDF.append(movieIdDF_temp, ignore_index=True)
    movieIdDF_temp.iloc[0,:]=""
                                                                               
movieIdDF = movieIdDF.sort_values(by="openDt")
movieIdDF

,directorpeopleNm,movieCd,movieNm,movieNmEn,openDt
0,장 피에르 다르덴,20201803,소년 아메드,Young Ahmed,
1,정진우,20201863,외아들,The Only Son,
2,정해지,20202001,수라,Sura,
37,이동준,20201963,가슴 큰 여동생,,
36,한제이,20209946,담쟁이,Take Me Home,
5,,20200601,1942: 언노운 배틀,Unknown Battle,
34,김태형,20191962,제8일의 밤,THE 8TH NIGHT,
33,박문칠,20200292,보드랍게,Comfort,
14,이보 반 아트,20201964,악플러 죽이기,The Columnist,
12,김정훈,20201965,해적: 도깨비 깃발,,


In [108]:
movieIdDF["year"]=movieIdDF["openDt"].apply(lambda x : str(x)[0:4])
movieIdDF["month"]=movieIdDF["openDt"].apply(lambda x : str(x)[4:6])
movieIdDF["day"]=movieIdDF["openDt"].apply(lambda x : str(x)[6:8])
movieIdDF

,directorpeopleNm,movieCd,movieNm,movieNmEn,openDt,year,month,day
0,장 피에르 다르덴,20201803,소년 아메드,Young Ahmed,,,,
1,정진우,20201863,외아들,The Only Son,,,,
2,정해지,20202001,수라,Sura,,,,
37,이동준,20201963,가슴 큰 여동생,,,,,
36,한제이,20209946,담쟁이,Take Me Home,,,,
5,,20200601,1942: 언노운 배틀,Unknown Battle,,,,
34,김태형,20191962,제8일의 밤,THE 8TH NIGHT,,,,
33,박문칠,20200292,보드랍게,Comfort,,,,
14,이보 반 아트,20201964,악플러 죽이기,The Columnist,,,,
12,김정훈,20201965,해적: 도깨비 깃발,,,,,


In [109]:
movieIdDF= movieIdDF.set_index("movieCd")
movieIdDF

,directorpeopleNm,movieNm,movieNmEn,openDt,year,month,day
movieCd,,,,,,,
20201803,장 피에르 다르덴,소년 아메드,Young Ahmed,,,,
20201863,정진우,외아들,The Only Son,,,,
20202001,정해지,수라,Sura,,,,
20201963,이동준,가슴 큰 여동생,,,,,
20209946,한제이,담쟁이,Take Me Home,,,,
20200601,,1942: 언노운 배틀,Unknown Battle,,,,
20191962,김태형,제8일의 밤,THE 8TH NIGHT,,,,
20200292,박문칠,보드랍게,Comfort,,,,
20201964,이보 반 아트,악플러 죽이기,The Columnist,,,,


In [110]:
for i in movieIdDF.index:
    movieInfoResultURL="http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieInfo.xml?key=9a8b54edfc3a767c4644a74ea2305ba8&movieCd="+i
    movieResultPage = urllib.request.urlopen(movieInfoResultURL)
    movieResultData = BeautifulSoup(movieResultPage, "html.parser")
    
    if movieResultData.find("typenm"):
        movieIdDF.at[i,"type"] = movieResultData.find("typenm").text
    movieIdDF.at[i,"actors"] = len(movieResultData.find_all("actor"))
    if movieResultData.find("actor"):
        movieIdDF.at[i,"peopleNm"] = movieResultData.find("actor").find("peoplenm").text
        movieIdDF.at[i,"cast"] = movieResultData.find("actor").find("cast").text
    else:
        movieIdDF.at[i,"peopleNm"]=""
        movieIdDF.at[i,"cast"]=""
        
movieIdDF

,directorpeopleNm,movieNm,movieNmEn,openDt,year,month,day,type,actors,peopleNm,cast
movieCd,,,,,,,,,,,
20201803,장 피에르 다르덴,소년 아메드,Young Ahmed,,,,,장편,1.0,클레어 보드손,
20201863,정진우,외아들,The Only Son,,,,,장편,3.0,최무룡,
20202001,정해지,수라,Sura,,,,,단편,0.0,,
20201963,이동준,가슴 큰 여동생,,,,,,장편,0.0,,
20209946,한제이,담쟁이,Take Me Home,,,,,장편,0.0,,
20200601,,1942: 언노운 배틀,Unknown Battle,,,,,장편,0.0,,
20191962,김태형,제8일의 밤,THE 8TH NIGHT,,,,,장편,5.0,이성민,
20200292,박문칠,보드랍게,Comfort,,,,,장편,0.0,,
20201964,이보 반 아트,악플러 죽이기,The Columnist,,,,,장편,1.0,카차 허버,펨케
